In [1]:
include("./src/REoptLite.jl")
using JuMP
using Xpress
using Main.REoptLite
using JSON


┌ Info: Xpress: Found license file /usr/local/opt/xpress/bin/xpauth.xpr
└ @ Xpress /Users/nlaws/.julia/packages/Xpress/kcAft/src/license.jl:42
┌ Info: Xpress: Development license detected.
└ @ Xpress /Users/nlaws/.julia/packages/Xpress/kcAft/src/license.jl:84


In [2]:
function run(scenario, fname)
    
    m = Model(optimizer_with_attributes(Xpress.Optimizer, "logfile" => "output.log"))
    results = run_reopt(m, REoptInputs(scenario))
    
    open(fname*".json", "w") do f
        JSON.print(f, results)
    end
    
    return results
end

run (generic function with 1 method)

In [3]:
scenario_template = JSON.parsefile("test/scenarios/STM_campus_BAU_existing_gen_sseb_crit.json")

Dict{String,Any} with 8 entries:
  "ElectricUtility" => Dict{String,Any}("outage_durations"=>Any[336],"outage_st…
  "ElectricTariff"  => Dict{String,Any}("urdb_label"=>"5e611fec5457a3665e019406…
  "Site"            => Dict{String,Any}("latitude"=>39.7407,"longitude"=>-105.1…
  "Generator"       => Dict{String,Any}("existing_kw"=>80.0,"only_runs_during_g…
  "ElectricLoad"    => Dict{String,Any}("loads_kw"=>Any[4323.64, 4308.39, 4295.…
  "PV"              => Dict{String,Any}("existing_kw"=>3580.54)
  "Storage"         => Dict{String,Any}("can_grid_charge"=>false)
  "Financial"       => Dict{String,Any}("owner_discount_pct"=>0.1,"offtaker_dis…

In [18]:
tzs = collect(range(17, stop=8760-336, step=336))

26-element Array{Int64,1}:
   17
  353
  689
 1025
 1361
 1697
 2033
 2369
 2705
 3041
 3377
 3713
 4049
 4385
 4721
 5057
 5393
 5729
 6065
 6401
 6737
 7073
 7409
 7745
 8081
 8417

In [17]:
crit_ld = scenario_template["ElectricLoad"]["critical_loads_kw"];

In [21]:
two_week_ld = Any[]

for tz in tzs
    push!(two_week_ld, crit_ld[tz:tz+335])
end

In [22]:
for ld in two_week_ld
    println(ld[end])
end

7.44062
12.4451
15.3369
10.0105
5.09326
10.1231
7.88971
12.2706
14.2894
9.76461
11.9615
12.8299
13.0251
8.94967
13.6465
13.0675
9.506
13.3915
10.7266
11.9795
12.7044
7.6517
11.8031
5.40665
7.35182
13.4861


In [24]:
maximum(sum(ld) for ld in two_week_ld)

3205.924210000001

In [4]:
results = Dict[]

for VoLL in range(100, stop=100)
    scenario_dict = copy(scenario_template)
    scenario_dict["ElectricUtility"]["outage_start_timesteps"] = collect(range(17, stop=8760-336, step=336))
    scenario_dict["Financial"]["VoLL"] = VoLL
    s = Scenario(scenario_dict)
    push!(results, run(s, "sseb_BAU_existing_gen"))
    println("run complete")
end

┌ Info: Checking URDB for 
│   urdb_label = 5e611fec5457a3665e019406
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\urdb.jl:151
┌ Info: Querying PVWatts for prodfactor with 
│   pv.name = PV
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:41
┌ Info: PVWatts success.
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:49
┌ Info: ("p.mg_tech_sizes_equal_grid_sizes", false)
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\constraints\outage_constraints.jl:81
┌ Info: Model built. Optimizing...
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:265
┌ Info: REopt solved with 
│   termination_status(m) = OPTIMAL
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:274


run complete


In [5]:
r = results

1-element Array{Dict,1}:
 Dict{String,Any}("net_capital_costs" => 1.18901678e6,"lcc" => 5.5978878e7,"PV_net_fixed_om_costs" => 515320.0,"year_one_energy_cost" => 1.16720545e6,"year_one_energy_produced_PV" => 5.948741e6,"year_one_fixed_cost" => 0.0,"total_demand_cost" => 2.668609423e7,"gen_net_variable_om_costs" => 0.0,"gen_total_fuel_cost" => 0.0,"year_one_min_charge_adder" => 0.0…)